In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime as dt
import itertools as it
import ast

In [2]:
def Excel_Date(date1):
    temp = dt.datetime(1899,12,30)
    delta = date1 - temp
    return float(delta.days)
def revert_Excel_Date(excel_date):
    # Excel dates start from December 30, 1899
    temp = dt.datetime(1899, 12, 30)
    # Convert the Excel date back to a datetime object
    reverted_date = temp + dt.timedelta(days=excel_date)
# Convert to pandas Timestamp
    pandas_date = pd.Timestamp(reverted_date)
# Format as YYYY-MM-DD
    formatted_date = pandas_date.strftime('%Y-%m-%d')
    return formatted_date

def performance_summary(df_performance, days):
    df_stats = []
    interval_days = df_performance.index[-1] - df_performance.index[0]
    for columns in df_performance.columns:
        portfolio = (1 + df_performance[columns]).cumprod()
        annualized_return = (portfolio.iloc[-1]) ** (365 / interval_days.days) - 1
        standard_deviation = df_performance[columns].std() * np.sqrt(252/days)
        less_than_zero = df_performance[columns][df_performance[columns] < 0]
        downside_deviation = less_than_zero.std() * np.sqrt(252/days)
        df_drawdown = ((portfolio - portfolio.cummax())/portfolio.cummax())
        max_drawdown = df_drawdown.min()
        df_stats.append({"Group": columns, "Annualized Return": round(annualized_return*100, 2), "Standard Deviation": round(100*standard_deviation, 2), "Downside Deviation": round(100*downside_deviation, 2), "Max Drawdown": round(100*max_drawdown, 2)})
    df_stats = pd.DataFrame(df_stats)
    return df_stats

In [3]:
df_score_summary = pd.read_csv("Score_Summary.csv")
df_score_summary

,Date,Ticker,Slope,Cheapness,RSQ weighted Reg Price,RSQ weighted Nominal Slope
0,45728.0,ACWV US Equity,0.002667,0.073082,68.577102,0.036785
1,45728.0,AGG US Equity,0.001507,0.015339,48.787598,0.018461
2,45728.0,AMLP US Equity,0.003447,0.052378,56.315528,0.035673
3,45728.0,ARKG US Equity,-0.007710,0.036419,3.718202,-0.019219
4,45728.0,ARKK US Equity,-0.010676,0.738046,28.896317,-0.054632
...,...,...,...,...,...,...
166515,43097.0,XLU US Equity,-0.001709,0.168841,26.867147,-0.009068
166516,43097.0,XLV US Equity,0.002817,0.039804,49.636137,0.023971
166517,43097.0,XLY US Equity,0.008111,-0.123398,80.886475,0.081187
166518,43097.0,XME US Equity,0.010408,-0.280024,17.100188,0.037145


In [4]:
def SwapList_Generator(df_scores, cur_date, tickers_held, swap_threshold):
    all_tickers = df_scores.loc[cur_date].dropna().index
    available_tickers = list(set(all_tickers) - set(tickers_held))
    #print(pd.DataFrame(df_scores.loc[cur_date][tickers_held]))
    df_held_scores = pd.DataFrame(df_scores.loc[cur_date][tickers_held]).sort_values(by = cur_date, ascending = True)
    df_available_scores = pd.DataFrame(df_scores.loc[cur_date][available_tickers]).sort_values(by = cur_date, ascending = False)
    df_max_swaps = df_available_scores.iloc[:len(tickers_held)]
    df_max_swaps.reset_index(inplace = True)
    df_held_scores.reset_index(inplace = True)
    #print(df_max_swaps)
    #print(df_held_scores)
    df_max_swaps["Existing Ticker"] = df_held_scores["Ticker"]
    df_max_swaps["Current Score"] = df_held_scores[cur_date]
    #print(df_max_swaps)
    df_max_swaps.set_index("Ticker", inplace = True)
    df_max_swaps["Score Difference"] = df_max_swaps[cur_date] - df_max_swaps["Current Score"]
    #print(df_max_swaps)
    df_swaps = df_max_swaps[df_max_swaps["Score Difference"] >= swap_threshold]
    return df_swaps

def Portfolio_Tracker(df_scores, df_price, date_range, num_positions, swap_threshold, trade_threshold, initial_capital = 1000):
    df_pct_change = df_price.sort_index(ascending = True).pct_change().dropna(how = "all")
    df_holdings = []
    tickers_held = df_scores.loc[date_range[0]].nlargest(num_positions).index
    df_weights = [1/num_positions] * num_positions
    portval = initial_capital
    Trade_tracker = True
    tickers_swapped_count = 0
    for index, date in enumerate(date_range[1:]):
        tickers_held = sorted(tickers_held)
        pct_changes = df_pct_change.loc[date, tickers_held]
        df_weights = [weight * (1 + pct_change) for weight, pct_change in zip(df_weights, pct_changes)]
        sum_weights = sum(df_weights)
        df_weights = [weight/sum_weights for weight in df_weights]
        portval = sum_weights * portval
        swaplist = SwapList_Generator(df_scores, date, tickers_held, swap_threshold)
        print(date)
        if swaplist is not None:
            max_score_diff = swaplist["Score Difference"].max()
            if max_score_diff >= trade_threshold:
                tickers_swapped = swaplist["Existing Ticker"].tolist()
                #print(tickers_swapped)
                tickers_held = list(set(tickers_held) - set(tickers_swapped))
                #print(tickers_held)
                tickers_added = swaplist.index.tolist()
                #print(tickers_added)
                tickers_held = list(set(tickers_held) | set(tickers_added))
                #print(tickers_held)
                tickers_held = sorted(tickers_held)
                df_weights = [1/num_positions] * num_positions
                Trade_tracker = True
                tickers_swapped_count = len(tickers_swapped)
        df_holdings.append({"Date": date, "Tickers": tickers_held, "Portfolio Value": portval, "Weights": df_weights, "Trade Tracker": Trade_tracker, "Number of Swaps": tickers_swapped_count})
        Trade_tracker = False
        tickers_swapped_count = 0
    df_holdings = pd.DataFrame(df_holdings)
    return df_holdings

In [5]:
df_price_all = pd.read_csv('Total Return PriceFile_Backfilled.csv', index_col = 0)
df_price_all.sort_index(ascending = True, inplace = True)
df_price_all

,ACWV US Equity,AGG US Equity,AMLP US Equity,ARKG US Equity,ARKK US Equity,ARKQ US Equity,ASHR US Equity,BBAX US Equity,BNDX US Equity,BOTZ US Equity,...,XLE US Equity,XLF US Equity,XLI US Equity,XLK US Equity,XLP US Equity,XLU US Equity,XLV US Equity,XLY US Equity,XME US Equity,XOP US Equity
DATE,,,,,,,,,,,,,,,,,,,,,
42734.0,72.610000,108.060000,63.000000,16.926875,20.050000,21.962875,23.450000,NaN,54.290000,15.000000,...,75.320000,23.250000,62.220000,48.360000,51.710000,48.570000,68.940000,81.400000,30.410000,165.680000
42738.0,72.800000,108.170000,63.700000,17.189688,20.436625,22.240000,23.890000,NaN,54.100000,15.100000,...,76.170000,23.510000,62.590000,48.790000,51.900000,48.450000,69.840000,81.880000,31.090000,168.040000
42739.0,73.280000,108.200000,63.900000,18.149875,21.359875,22.650000,24.470000,NaN,54.130000,15.320000,...,76.010000,23.700000,62.960000,48.960000,51.900000,48.630000,70.390000,82.970000,32.230000,168.840000
42740.0,73.670000,108.650000,64.000000,17.926687,21.150000,22.490000,24.660000,NaN,54.090000,15.350000,...,75.820000,23.460000,62.780000,49.040000,52.070000,48.680000,70.750000,82.910000,32.660000,168.320000
42741.0,73.640000,108.290000,64.050000,18.230000,21.365000,22.650000,24.250000,NaN,53.950000,15.370000,...,75.890000,23.540000,63.140000,49.400000,52.120000,48.830000,70.950000,83.320000,31.800000,167.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45722.0,137.368296,122.103584,99.497391,25.103877,56.079290,73.594408,29.972842,66.154389,61.491105,32.003721,...,121.912486,57.601104,152.939273,235.244093,102.407196,98.068617,170.563417,223.598331,63.146116,140.642262
45723.0,138.407336,121.954784,100.880413,25.222801,56.412711,74.348623,29.741774,65.957071,61.491105,32.293161,...,123.997309,57.297818,154.725678,238.651212,102.741981,99.890925,170.574843,223.185968,62.720588,143.046603
45726.0,137.631042,122.537587,101.221157,23.774085,51.475927,70.705020,29.389670,64.773158,61.503786,31.001021,...,124.919169,56.014683,152.305388,228.506297,102.518791,100.989479,168.735176,215.220854,60.257005,143.023260


In [6]:
df_price_list_copy = []
df_score_summary["Cur Price"] = df_score_summary.apply(lambda x: df_price_all.loc[x["Date"], x["Ticker"]], axis = 1)

In [7]:
df_score_summary["Combined Term 2"] = df_score_summary["RSQ weighted Nominal Slope"] / df_score_summary["Cur Price"]

In [8]:
df_score_summary

,Date,Ticker,Slope,Cheapness,RSQ weighted Reg Price,RSQ weighted Nominal Slope,Cur Price,Combined Term 2
0,45728.0,ACWV US Equity,0.002667,0.073082,68.577102,0.036785,135.600733,0.000271
1,45728.0,AGG US Equity,0.001507,0.015339,48.787598,0.018461,121.743982,0.000152
2,45728.0,AMLP US Equity,0.003447,0.052378,56.315528,0.035673,101.962778,0.000350
3,45728.0,ARKG US Equity,-0.007710,0.036419,3.718202,-0.019219,24.833594,-0.000774
4,45728.0,ARKK US Equity,-0.010676,0.738046,28.896317,-0.054632,53.358143,-0.001024
...,...,...,...,...,...,...,...,...
166515,43097.0,XLU US Equity,-0.001709,0.168841,26.867147,-0.009068,54.384386,-0.000167
166516,43097.0,XLV US Equity,0.002817,0.039804,49.636137,0.023971,84.465437,0.000284
166517,43097.0,XLY US Equity,0.008111,-0.123398,80.886475,0.081187,100.563133,0.000807
166518,43097.0,XME US Equity,0.010408,-0.280024,17.100188,0.037145,36.876187,0.001007


In [ ]:
df_combo_dictt = []
df_performance_summary_tracker = []
for days_coefficient in [0, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150]:
    for swap_threshold in [0.002, 0.005, 0.01, 0.015]:
        for trade_threshold in [0.015, 0.02, 0.03, 0.04, 0.05]:
            df_score_summary_current = df_score_summary.copy()
            df_score_summary_current["Combined Score"] = df_score_summary_current["Cheapness"] + days_coefficient * df_score_summary_current["Combined Term 2"]
            df_scorefile = df_score_summary_current.pivot_table(index = "Date", columns = "Ticker", values = "Combined Score")
            date_range = df_scorefile.index
            
            # Drop the 3 ARK tickers
            df_scorefile.drop(columns = ["ARKQ US Equity", "ARKG US Equity", "ARKK US Equity", "AGG US Equity", "BNDX US Equity", "MBB US Equity", "PCY US Equity", "SCHP US Equity", "SLQD US Equity", "VCIT US Equity"], inplace = True)
            df_scorefile = df_scorefile.dropna(axis = 1, how = "all")
            df_holdings = Portfolio_Tracker(df_scorefile, df_price_all, date_range, 8, swap_threshold, trade_threshold)
            df_combo_dictt.append({"Days Coefficient": days_coefficient, "Swap Threshold": swap_threshold, "Trade Threshold": trade_threshold, "Holdings Summary": df_holdings})
            df_holdings_copy = df_holdings.copy()
            df_holdings_copy.set_index("Date", inplace = True)
            df_holdings_copy.index = df_holdings_copy.index.map(revert_Excel_Date)
            df_holdings_copy.index = pd.to_datetime(df_holdings_copy.index)
            df_perf = performance_summary(df_holdings_copy[["Portfolio Value"]], 1)
            df_performance_summary_tracker.append({"Days Coefficient": days_coefficient, "Swap Threshold": swap_threshold, "Trade Threshold": trade_threshold, "Performance Summary": df_perf})

In [ ]:
df_combo_dict = pd.DataFrame(df_combo_dictt)

In [ ]:
df_combo_dict

In [ ]:
df_score_summary

In [ ]:
performance_summary(df_holdings_copy[["Portfolio Value"]].pct_change().dropna(), 1).loc[0, :]

In [ ]:
df_combo_dict.loc[0, "Holdings Summary"]

In [ ]:
perf_combo_tracker = []
for index, row in df_combo_dict.iterrows():
    df_holdings_copy = row["Holdings Summary"]
    #df_holdings_copy.set_index("Date", inplace = True)
    #df_holdings_copy.index = df_holdings_copy.index.map(revert_Excel_Date)
    #df_holdings_copy.index = pd.to_datetime(df_holdings_copy.index)
    df_perf = performance_summary(df_holdings_copy[["Portfolio Value"]].pct_change(), 1)
    perf_combo_tracker.append({"Days Coefficient": row["Days Coefficient"], "Swap Threshold": row["Swap Threshold"], "Trade Threshold": row["Trade Threshold"], "Annualized Return": df_perf.loc[0, "Annualized Return"], "Standard Deviation": df_perf.loc[0, "Standard Deviation"], "Downside Deviation": df_perf.loc[0, "Downside Deviation"], "Max Drawdown": df_perf.loc[0, "Max Drawdown"], "Swap Count": df_holdings_copy["Number of Swaps"].sum(), "Trade Days Count": len(df_holdings_copy[df_holdings_copy["Trade Tracker"]==True])})
df_perf_combo_tracker = pd.DataFrame(perf_combo_tracker)

In [ ]:
df_perf_combo_tracker

In [ ]:
df_swap_by_Return = df_perf_combo_tracker.sort_values(by = "Annualized Return", ascending = False)
df_swap_by_Return.set_index("Days Coefficient", inplace = True)
df_swap_by_Return

In [ ]:
df_swap_by_Return.to_csv("df_swap_by_Return.csv")

In [ ]:
plt.scatter(df_swap_by_Return.index, df_swap_by_Return["Annualized Return"])

In [ ]:
plt.figure(figsize = (12, 8))
plt.scatter(df_swap_by_Return["Swap Count"], df_swap_by_Return["Annualized Return"], c = df_swap_by_Return.index, cmap = "viridis")
plt.colorbar(label = "Days Coefficient")
plt.xlabel("Swap Count")
plt.ylabel("Annualized Return")
plt.title("Return vs Swap Count")
plt.show()

In [ ]:
plt.figure(figsize = (12, 8))
plt.scatter(df_swap_by_Return.index, df_swap_by_Return["Annualized Return"], c = df_swap_by_Return["Max Drawdown"], cmap = "viridis")
plt.colorbar(label = "Max Drawdown")
plt.xlabel("Days Coefficient")
plt.ylabel("Annualized Return")
plt.title("Return vs Days Coefficient")
plt.show()

In [ ]:
df_swap_by_Return_100Days_coefficient_subset = df_swap_by_Return.loc[100, :]
plt.scatter(df_swap_by_Return_100Days_coefficient_subset["Swap Count"], df_swap_by_Return_100Days_coefficient_subset["Annualized Return"], c = df_swap_by_Return_100Days_coefficient_subset["Swap Threshold"], cmap = "viridis")
plt.colorbar(label = "Swap Threshold")
plt.xlabel("Swap Count")
plt.ylabel("Annualized Return")
plt.title("Return vs Swap Count")
plt.show()

In [ ]:
df_swap_by_Return

In [ ]:
df_perf_combo_tracker.loc[df_perf_combo_tracker["Annualized Return"].idxmax()]

In [ ]:
# Create an interactive chart with Days Coefficient, Swap Threshold & Trade Threshold as the draggable variables
import plotly.express as px
fig = px.scatter_3d(df_perf_combo_tracker, x='Days Coefficient', y='Swap Threshold', z='Trade Threshold', color='Annualized Return', size='Standard Deviation', hover_data=['Downside Deviation', 'Max Drawdown'])
fig.update_layout(
    width=1200,  # Increase the width (make it longer)
    height=600   # You can also adjust the height to maintain aspect ratio
)
fig.show(figsize = (20, 5))

In [ ]:
# Create an interactive chart with Days Coefficient, Swap Threshold & Trade Threshold as the draggable variables
import plotly.express as px
fig = px.scatter_3d(df_perf_combo_tracker, x='Days Coefficient', y='Swap Threshold', z='Trade Threshold', color='Max Drawdown', size='Standard Deviation', hover_data=['Downside Deviation', 'Max Drawdown'])
fig.update_layout(
    width=1200,  # Increase the width (make it longer)
    height=600   # You can also adjust the height to maintain aspect ratio
)
fig.show(figsize = (20, 5))

In [ ]:
df_perf_combo_tracker["Annualized Return"].idxmax()

In [ ]:
df_combo_dict.loc[162, "Holdings Summary"][["Portfolio Value"]].plot()

In [ ]:
df_comp_best_model = df_combo_dict.loc[162, "Holdings Summary"]

In [ ]:
df_stock_model_comp = pd.read_csv("Stock Model Return.csv", index_col = 0)
df_stock_model_comp.index = pd.to_datetime(df_stock_model_comp.index)
df_stock_model_comp

In [ ]:
df_comp_best_model = df_comp_best_model[["Tickers", "Portfolio Value"]].copy()

In [ ]:
df_comp_best_model["SPX Index Value"] = df_stock_model_comp["SPX Index Value"]
df_comp_best_model["Stock Model Value"] = df_stock_model_comp["Portval"]

In [ ]:
df_comp_best_model.fillna(method = "ffill", inplace = True)

In [ ]:
df_comp_best_model.dropna(inplace = True)
df_comp_best_model = df_comp_best_model.loc[:"2024-11-12"]

In [ ]:
df_comp_best_model["Portfolio Value"] = df_comp_best_model["Portfolio Value"]/df_comp_best_model["Portfolio Value"].iloc[0]
df_comp_best_model["SPX Index Value"] = df_comp_best_model["SPX Index Value"]/df_comp_best_model["SPX Index Value"].iloc[0]
df_comp_best_model["Stock Model Value"] = df_comp_best_model["Stock Model Value"]/df_comp_best_model["Stock Model Value"].iloc[0]

In [ ]:
df_comp_best_model[["Portfolio Value", "SPX Index Value", "Stock Model Value"]].plot()

In [ ]:
performance_summary((df_comp_best_model[["Portfolio Value", "SPX Index Value", "Stock Model Value"]].pct_change()), 1)

In [ ]:
df_comp_best_model[["Portfolio Value"]].plot(figsize = (20, 10))

In [ ]:
df_price_all_copy = df_price_all.copy()
df_price_all_copy.index = df_price_all_copy.index.map(revert_Excel_Date)
df_price_all_copy.index = pd.to_datetime(df_price_all_copy.index)
df_price_all_copy_pct = df_price_all_copy.pct_change().dropna()
df_price_all_copy_pct

In [ ]:
df_perf_summary_ETFs = performance_summary(df_price_all_copy_pct, 1)
df_perf_summary_ETFs.set_index("Group", inplace = True)
df_perf_summary_ETFs

In [ ]:
df_perf_summary_ETFs

In [ ]:
df_perf_summary_ETFs.describe()

In [ ]:
df_perf_summary_ETFs.idxmax()

In [ ]:
df_combo_dict.loc[162, "Holdings Summary"][["Portfolio Value"]].loc["2024-12-31":].plot()

In [ ]:
df_perf_summary_ETFs.loc["HEFA US Equity"]

In [ ]:
df_perf_summary_ETFs.loc["EEMV US Equity"]

In [ ]:
df_perf_summary_ETFs

In [ ]:
df_combo_dict.loc[162, "Holdings Summary"]

In [ ]:
(df_combo_dict.loc[162, "Holdings Summary"]["Number of Swaps"].sum()/8)/((df_combo_dict.loc[162, "Holdings Summary"].index[-1] - df_combo_dict.loc[162, "Holdings Summary"].index[0]).days)*365

In [ ]:
# Still once a week rebalancing

In [ ]:
df_target_trade_tracker = df_combo_dict.loc[162, "Holdings Summary"].copy()

In [ ]:
df_target_trade_tracker[df_target_trade_tracker["Trade Tracker"] == True]